# Provingground - HoTT

These notes concern the object _HoTT_, which has the core implementation of homotopy type theory. Implementation details are (rather, will be) in the [scaladocs](http://siddhartha-gadgil.github.io/ProvingGround/).

The major components of homotopy type theory implemented in the object HoTT are

* Terms, types and Universes.
* Function and dependent function types.
* λs.
* Pairs and Dependent pairs.
* Disjoint union types.
* Types 0 and 1 and an object in the latter.
* Identity types

Inductive types, induction and recursion are in different objects as they are rather subtle. The other major way (also not in the _HoTT_ object) of constructing non-composite types is to wrap scala types, possibly including symbolic algebra.


The _core_ project contains code that is agnostic to how it is run. In particular this also compiles to scala-js.

In [1]:
import $ivy.`in.ac.iisc.math::core-jvm:0.1-SNAPSHOT`

import $ivy.$                                       

### Universes, Symbolic types

We have a family of universes, but mostly use the first one denoted by Type. Given a type, we can construct symbolic objects of that type. We construct such a type _A_.

In [2]:
import provingground._
repl.pprinter.bind(translation.FansiShow.fansiPrint)
import HoTT._
val A ="A" :: Type
A == Type.::("A")

import provingground._

import HoTT._

A: Typ[Term] = A
res1_4: Boolean = true

We consider a symbolic object of the type _A_

In [3]:
val a ="a" :: A

a: Term = a

## Function types, lambdas, Identity

Given types A and B, we have the function type A → B. An element of this is a function from A to B. 

We can construct functions using λ's. Here, for the type _A_, we construct the identity on _A_ using a lambda. We can then view this as a dependent function of _A_, giving the identity function.

In this definition, two λ's are used, with the method _lmbda_ telling the TypecompilerType that the result is a (non-dependent) function. 

In [4]:
val id = lambda(A)(lmbda(a)(a))

id: FuncLike[Typ[Term], Func[Term, Term]] = (A : 𝒰 ) ↦ (a : A) ↦ a

The type of the identity function is a mixture of Pi-types and function types. Which of these to use is determined by checking dependence of the type of the value on the varaible in a λ-definition.

In [5]:
id.typ
lmbda(a)(a).typ
lmbda(a)(a).typ.dependsOn(A)

res4_0: Typ[FuncLike[Typ[Term], Func[Term, Term]]] = ∏(A : 𝒰 ){ (A → A) }
res4_1: Typ[Func[Term, Term]] = (A → A)
res4_2: Boolean = true

The lambdas have the same effect at runtime. It is checked if the type of the value depends on the variable.
The result is either _LambdaFixed_ or _Lambda_ accordingly.

In [6]:
val indep = lmbda(a)(a)
val dep = lambda(a)(a)
indep == dep

indep: Func[Term, Term] = (a : A) ↦ a
dep: FuncLike[Term, Term] = (a : A) ↦ a
res5_2: Boolean = true

### Hygiene for λs

A new variable object, which has the same toString, is created in making lambdas. This is to avoid name clashes.

In [7]:
val l = dep.asInstanceOf[LambdaFixed[Term, Term]]
l.variable
l.variable == a

l: LambdaFixed[Term, Term] = (a : A) ↦ a
res6_1: Term = a
res6_2: Boolean = false

## Modus Ponens

We construct Modus Ponens, as an object in Homotopy Type theory. Note that A ->: B is the function type A → B.

In [8]:
val B = "B" :: Type

val f = "f" :: (A ->: B)

val mp = lambda(A)(lambda(B)(lmbda(a)(lmbda(f)(f(a)))))

B: Typ[Term] = B
f: Func[Term, Term] = f
mp: FuncLike[Typ[Term], FuncLike[Typ[Term], Func[Term, Func[Func[Term, Term], Term]]]] = (A : 𝒰 ) ↦ (B : 𝒰 ) ↦ (a : A) ↦ (f : (A → B)) ↦ f(a)

The type of Modus Ponens is again a mixture of Pi-types and function types.

In [9]:
mp.typ

res8: Typ[FuncLike[Typ[Term], FuncLike[Typ[Term], Func[Term, Func[Func[Term, Term], Term]]]]] = ∏(A : 𝒰 ){ ∏(B : 𝒰 ){ (A → ((A → B) → B)) } }

We can apply modus ponens with the roles of _A_ and _B_ reversed. This still works because variable clashes are avoided.

In [10]:
val mpBA = mp(B)(A)
mpBA.typ == B ->: (B ->: A) ->: A

mpBA: Func[Term, Func[Func[Term, Term], Term]] = (a : B) ↦ (f : (B → A)) ↦ f(a)
res9_1: Boolean = true

### Equality of λs

Lambdas do not depend on the name of the variable.

In [11]:
val aa = "aa" :: A
lmbda(aa)(aa) == lmbda(a)(a)
(lmbda(aa)(aa))(a) == a

aa: Term = aa
res10_1: Boolean = true
res10_2: Boolean = true

## Dependent types

Given a type family, we can construct the corresponding Pi-types and Sigma-types. We start with a formal type family, which is just a symbolic object of the appropriate type.

In [12]:
val Bs = "B(_ : A)" :: (A ->: Type)

Bs: Func[Term, Typ[Term]] = B(_ : A)

### Pi-Types

In addition to the case class constructor, there is an agda/shapeless-like  convenience method for constructing Pi-types. Namely, given a type expression that depends on a varaible _a : A_, we can construct the Pi-type correspoding to the obtained λ-expression.

Note that the !: method just claims and checks a type, and is useful (e.g. here) for documentation.

In [13]:
val fmly = (a !: A) ~>: (Bs(a) ->: A)

fmly: GenFuncTyp[Term, Func[Term, Term]] = ∏(a : A){ (B(_ : A)(a) → A) }

### Sigma-types

There is also a convenience method for defining Sigma types using λs.

In [14]:
Sgma(a !: A, Bs(a))

res13: SigmaTyp[Term, Term] = ∑(a : { B(_ : A)(a) }

In [15]:
Sgma(a !: A, Bs(a) ->: Bs(a) ->: A)

res14: SigmaTyp[Term, Func[Term, Func[Term, Term]]] = ∑(a : { (B(_ : A)(a) → (B(_ : A)(a) → A)) }

## Pair types

Like functions and dependent functions, pairs and dependent pairs can be handled together. The _mkPair_ function assignes the right type after checking dependence, choosing between pair types, pairs and dependent pairs.

In [16]:
val ba = "b(a)" :: Bs(a)
val b = "b" :: B
mkPair(A, B)
mkPair(a, b)
mkPair(a, b).typ
mkPair(a, ba).typ

ba: Term = b(a)
b: Term = b
res15_2: AbsPair[Term, Term] = A×B
res15_3: AbsPair[Term, Term] = (a , b)
res15_4: Typ[U] = A×B
res15_5: Typ[U] = ∑(a : { B(_ : A)(a) }

In [17]:
mkPair(A, B).asInstanceOf[ProdTyp[Term, Term]]

res16: ProdTyp[Term, Term] = A×B

## Plus types

We can also construct the plus type _A plus B_, which comes with two inclusion functions.

In [18]:
val AplusB = PlusTyp(A, B)

AplusB: PlusTyp[Term, Term] = A + B

In [19]:
AplusB.incl1(a)

res18: PlusTyp.FirstIncl[Term, Term] = FirstIncl(PlusTyp(A,B),a)

In [20]:
AplusB.incl2

res19: Func[Term, PlusTyp.ScndIncl[Term, Term]] = ($gr : B) ↦ ScndIncl(PlusTyp(A,B),$gr)

In the above, a λ was used, with a variable automatically generated. These have names starting with $ to avoid collision with user defined ones.

## Identity type

We have an identity type associated to a type _A_, with reflexivity giving terms of this type.

In [21]:
val eqAa = IdentityTyp(A, a, a)
val ref = Refl(A, a)

eqAa: IdentityTyp[Term] = a = a
ref: Refl[Term] = Refl(A,a)

In [22]:
ref.typ == eqAa

res21: Boolean = true

## The Unit and the  Nought

Finally, we have the types corresponding to _True_ and _False_

In [23]:
Unit
Zero
Star !: Unit

res22_0: Unit.type = Unit
res22_1: Zero.type = Zero
res22_2: Term = Star